In [1]:
import torch
import torch.nn.functional as F

## MSE

In [2]:
x = torch.tensor([0.1, 0.2, 0.3, 0.4, 0.5], requires_grad=True)
y = torch.tensor([0.14, 0.18, 0.33, 0.49, 0.51])

loss = torch.nn.MSELoss(reduction="mean")
mse = loss(x, y)
print(mse.detach())

loss = torch.nn.MSELoss(reduction="sum")
mse = loss(x, y)
print(mse.detach())

tensor(0.0022)
tensor(0.0111)


In [3]:
sqaure_err = torch.square(x-y)

mse = torch.mean(sqaure_err)
print(mse.detach())

mse = torch.sum(sqaure_err)
print(mse.detach())

tensor(0.0022)
tensor(0.0111)


## MAE

In [4]:
loss = torch.nn.L1Loss(reduction="mean")
mae = loss(x, y)
print(mae.detach())

loss = torch.nn.L1Loss(reduction="sum")
mae = loss(x, y)
print(mae.detach())

tensor(0.0380)
tensor(0.1900)


In [5]:
l1_err = ...

mae = torch.mean(l1_err)
print(mae.detach())

mae = torch.sum(l1_err)
print(mae.detach())

tensor(0.0380)
tensor(0.1900)


## HuberLoss

In [6]:
loss = torch.nn.HuberLoss(reduction='mean', delta=1.0)
huber = loss(x, y)
print(huber.detach())

loss = torch.nn.HuberLoss(reduction='sum', delta=1.0)
huber = loss(x, y)
print(huber.detach())

tensor(0.0011)
tensor(0.0055)


In [7]:
def my_huber(x, y, delta=1, reduction="mean"):
    huber_l2 = ...
    huber_l1 = ...
    condition = ...

    t = torch.where(condition, huber_l1, huber_l2)
    
    if reduction == "mean":
        return torch.mean(t)
    elif reduction == "sum":
        return torch.sum(t)
    else:
        raise ValueError(f"Invalid arg: {reduction}, only support mean, sum")

huber = my_huber(x, y, reduction="mean")
print(huber.detach())

huber = my_huber(x, y, reduction="sum")
print(huber.detach())

tensor(0.0011)
tensor(0.0055)


## Cross Entropy

In [8]:
x = torch.tensor(
    [[-1, 2.6, 0.1, -0.04, 0.36],
    [-0.065, 0, 1.8, 0.1, -10.05]], requires_grad=True
)
y = torch.tensor([1, 2])

loss = torch.nn.CrossEntropyLoss(weight=None, ignore_index=-100, reduction='mean')
ce = loss(x, y)
print(ce.detach())

loss = torch.nn.CrossEntropyLoss(reduction='sum')
ce = loss(x, y)
print(ce.detach())

tensor(0.3299)
tensor(0.6599)


In [9]:
def my_ce(x, y, reduction="mean"):
    batch_size, num_cls = x.shape
    y = F.one_hot(y, num_classes=num_cls)

    ce = ... # hint: torch.log_softmax

    if reduction == "mean":
        return torch.sum(ce) / batch_size
    elif reduction == "sum":
        return torch.sum(ce)
    else:
        raise ValueError(f"Invalid arg: {reduction}, only support mean, sum")

ce = my_ce(x, y, reduction="mean")
print(ce.detach())

ce = my_ce(x, y, reduction="sum")
print(ce.detach())

tensor(0.3299)
tensor(0.6599)


## Focal Loss

In [10]:
class FocalLoss(torch.nn.Module):
    def __init__(self, gamma=2, alpha=[0.1, 0.3], size_average=True):
        super(MyFocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.size_average = size_average

    def forward(self, x, y):
        batch_size, num_cls = x.shape
        y = F.one_hot(y, num_classes=num_cls)

        softmax_x = torch.softmax(x, dim=1)
        softmax_x = torch.sum(softmax_x * y, 1)

        at = torch.tensor(self.alpha)
        loss = -1*at * (1-softmax_x)**self.gamma * torch.log(softmax_x)
        
        if self.size_average:
            return torch.mean(loss)
        else:
            return torch.sum(loss)


loss = FocalLoss()
focal = loss(x, y)
print(focal.detach())

# Eqauls Cross Entro
loss = FocalLoss(gamma=0, alpha=[1, 1])
focal = loss(x, y)
print(focal.detach())

tensor(0.0075)
tensor(0.3299)


## Center Loss

In [11]:
class CenterLoss(torch.nn.Module):
    """
    Ref: https://github.com/KaiyangZhou/pytorch-center-loss/blob/master/center_loss.py
    """
    def __init__(self, num_classes=5, feat_dim=2):
        super(CenterLoss, self).__init__()
        self.num_classes = num_classes
        self.feat_dim = feat_dim
        self.centers = torch.nn.Parameter(torch.randn(self.num_classes, self.feat_dim))

    def forward(self, x, y):
        """
        Args:
            x: feature matrix with shape (batch_size, feat_dim).
            y: ground truth labels with shape (num_classes).
        """
        batch_size, _ = x.shape
        distmat = torch.pow(x, 2).sum(dim=1, keepdim=True).expand(batch_size, self.num_classes) + \
                  torch.pow(self.centers, 2).sum(dim=1, keepdim=True).expand(self.num_classes, batch_size).t()
        distmat.addmm_(1, -2, x, self.centers.t())
        classes = torch.arange(self.num_classes).long()

        y = y.unsqueeze(1).expand(batch_size, self.num_classes)
        mask = y.eq(classes.expand(batch_size, self.num_classes))
        
        dist = distmat * mask.float()
        loss = dist.clamp(min=1e-12, max=1e+12).sum() / batch_size
        
        return loss

embedding = torch.tensor([[-4.3, 0.26],[3.38, 0.071]], requires_grad=True)
y = torch.tensor([1, 2])

loss = CenterLoss()
center = loss(embedding, y)
print(center)

tensor(7.2386, grad_fn=<DivBackward0>)


## Triplet Loss

In [12]:
triplet_loss = torch.nn.TripletMarginLoss(margin=1.0, p=2)
anchor = torch.tensor([[-0.1205, -0.0747,  0.2110,  0.7640, -0.0889, -1.8335,  0.5339,  1.6246, -0.4688,  0.5528, -1.9838,  0.5563,  0.3497,  0.0260, -0.2148,  0.8602]], requires_grad=True)
positive = torch.tensor([[-0.7199, -0.2645,  0.2314, -1.0831,  1.6406, -1.6086,  0.2405, -0.3084,-0.0162,  1.1488, -0.9403,  0.2147, -1.5522, -0.0652, -0.0642, -0.5128]], requires_grad=True)
negative = torch.tensor([[-0.1581, -1.1795,  1.3480, -0.2423, -0.7676,  0.5932,  2.4350,  1.0902,0.6253, -0.5601, -0.2758,  1.2256, -1.0179,  0.2688, -0.4038,  0.9660]], requires_grad=True)

output = triplet_loss(anchor, positive, negative)
print(output.detach())

tensor(0.5966)
